In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv('M_data_modeling_561SMILES.csv')
data

,SMILES,ALogP,ALogp2,AMR,apol,nAtom,ATSm1,ATSm2,ATSm3,ATSm4,...,VAdjMat,MW,WTPT.1,WTPT.2,WTPT.3,WPATH,WPOL,XLogP,Zagreb,mean_zscore
0,[2H][C@@](C)(NC(=O)COC(F)F)c1cc(F)cc(Cl)c1COc2...,4.9035,24.044312,132.0162,71.093239,60,53.349434,47.297124,70.977093,73.804066,...,6.321928,533.930913,76.681182,2.017926,36.181243,4808,63,4.639,194,1.511929
1,[2H][C@@](N)(CO)C(O)=O,-1.4888,2.216525,22.1161,13.453551,14,9.690430,7.246378,9.368815,8.992028,...,3.584963,105.092711,14.271680,1.783960,11.777389,62,9,-3.956,26,-0.293797
2,[2H]C([2H])([2H])N(C([2H])([2H])[2H])C([2H])([...,2.2509,5.066551,62.1359,34.790688,31,16.564855,18.002201,24.671700,23.731145,...,5.000000,204.268694,48.532898,1.941316,32.890133,1278,48,1.633,76,0.148202
3,[2H]C1([2H])[C@H]2C[C@@H](Oc3ccc(cn3)C(F)(F)F)...,3.6800,13.542400,107.9118,60.714274,51,42.011121,41.986894,67.242763,64.111806,...,6.209453,458.409095,71.640211,2.046863,33.229469,3752,61,4.196,184,-0.944121
4,[2H]C1([2H])C([C@@H](CC#N)n2cc(cn2)-c3ncnc4[nH...,2.7411,7.513629,87.7190,48.522274,41,25.216222,28.693159,40.251216,37.588003,...,5.700440,306.365663,62.343269,2.011073,37.936323,2435,65,2.696,124,-0.076593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,OS(O)(=O)=O.C[C@H](N)Cc1ccccc1.C[C@H](N)Cc2ccccc2,1.9234,3.699468,101.1792,58.658204,53,34.945590,34.557962,35.311542,18.332364,...,5.584963,368.492698,47.570246,1.902810,16.789671,200000000068,18,1.760,108,0.446026
557,OS(O)(=O)=O.COc1ccc(cc1)S(=O)(=O)NCc2cc(CN3CCC...,1.7642,3.112402,141.8191,81.093755,72,56.531199,60.281187,69.859569,56.323915,...,6.285402,557.683107,73.720532,1.992447,35.620419,160000002987,46,0.840,188,-0.574772
558,OS(O)(=O)=O.N[C@H]1C[C@@H]1c2ccccc2.N[C@H]3C[C...,1.0414,1.084514,97.1628,55.991032,49,34.945590,36.557962,35.311542,20.332364,...,5.700440,364.460935,48.425994,1.937040,16.857489,200000000054,20,0.950,124,-1.809539
559,OS(O)(=O)=O.O[C@H]1CCN(C1)C(=O)Nc2cnn3ccc(nc23...,1.0149,1.030022,124.6342,68.389032,60,51.938537,53.573697,69.483914,57.391041,...,6.285402,526.515572,72.658877,2.018302,42.006338,155000002643,49,1.310,192,0.620174


In [2]:
# Split data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
# Define custom dataset class to handle data
class DescriptorDataset(Dataset):
    def __init__(self, data):
        self.x = data.drop(columns=["SMILES", "mean_zscore"]).values.astype(float)
        self.y = data["mean_zscore"].values.astype(float)
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])
    
# Define custom neural network
class Net(torch.nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create DataLoader objects for training and validation data
train_dataset = DescriptorDataset(train_data)
val_dataset = DescriptorDataset(val_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Define neural network model
net = Net(train_dataset.x.shape[1])

# Define optimizer and loss function
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# Train model
num_epochs = 20
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs.flatten(), labels.float())
        loss.backward()
        optimizer.step()

    # Compute validation loss
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = net(inputs.float())
            val_loss += criterion(outputs.flatten(), labels.float())
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")


Epoch 1, Train Loss: 60203961042337792.0000, Val Loss: 182551090925928448.0000
Epoch 2, Train Loss: 463963017516679168.0000, Val Loss: 443194445458833408.0000
Epoch 3, Train Loss: 394931210758914048.0000, Val Loss: 636796177999724544.0000
Epoch 4, Train Loss: 404636977854414848.0000, Val Loss: 154735216269721600.0000
Epoch 5, Train Loss: 12347207416020992.0000, Val Loss: 21311760866738176.0000
Epoch 6, Train Loss: 328472924782592.0000, Val Loss: 2293532937158656.0000
Epoch 7, Train Loss: 2618559687557120.0000, Val Loss: 21785551594061824.0000
Epoch 8, Train Loss: 38502590807277568.0000, Val Loss: 7163505086038016.0000
Epoch 9, Train Loss: 177480145043456.0000, Val Loss: 541328685400064.0000
Epoch 10, Train Loss: 3768611000811520.0000, Val Loss: 1893216618020864.0000
Epoch 11, Train Loss: 31717723209728.0000, Val Loss: 6459124007043072.0000
Epoch 12, Train Loss: 121751551868928.0000, Val Loss: 250816241860608.0000
Epoch 13, Train Loss: 194614094987264.0000, Val Loss: 106677021966336.000